## Working System:
Keeps all the tiles coming from the same slide within the same training or validation group.
Splits slides into training and validation groups and generates a validation file list.

__source:__

images/\
&emsp;B2020.xxxx image11.png\
&emsp;B2020.xxxx image12.png\
&emsp;B2019.xxxx image21.png\
&emsp;B2019.xxxx image22.png\
&emsp;B2018.xxx image33.png\
&emsp;B2018.xxx image34.png\
&emsp;.\
&emsp;.\
&emsp;.\
&emsp;\
labels/\
&emsp;B2020.xxxx image11-labelled.png\
&emsp;B2020.xxxx image12-labelled.png\
&emsp;B2019.xxxx image21-labelled.png\
&emsp;B2019.xxxx image21-labelled.png\
&emsp;B2018.xxx iamge33-labelled.png\
&emsp;B2018.xxx iamge34-labelled.png\
&emsp;.\
&emsp;.\
&emsp;.

__target:__

val.txt  

`image21.png`\
`image22.png`


In [1]:
from os import path
import os
import numpy as np
import shutil
import pandas as pd
import sys
from pathlib import Path

In [2]:
imageDir = "/scicore/home/pissal00/ercan0000/220310_training/GT/"
allFile = os.listdir(imageDir)

In [3]:
os.mkdir(os.path.join(imageDir, "labels"))
os.mkdir(os.path.join(imageDir, "images"))

In [4]:
for tile in allFile:
    if tile.endswith("-labelled.png"):
        shutil.move(os.path.join(imageDir, tile), os.path.join(imageDir, "labels", tile))
    elif tile.endswith(".png"):
        shutil.move(os.path.join(imageDir, tile), os.path.join(imageDir, "images", tile))

In [5]:
output = os.path.join(imageDir, "images.txt")
Path(output).touch()

allImages = os.listdir(os.path.join(imageDir,'images'))

orig_stdout = sys.stdout
f = open(output, 'w')
sys.stdout = f

for image in allImages:
    #name , extension = path.splitext(image)
    print(image)

sys.stdout = orig_stdout
f.close()

In excel I will extract the list of 98 slides.  
`wsi.txt` will be the output.

In [6]:
from fastcore.xtras import Path

In [7]:
path  = Path(imageDir+'wsi.txt')

allWSI = (path).read_text().split('\n')
allWSI[:10]

['b2012.5967_CD3',
 'b2013.27504_CD3',
 'B2011.24261_CD3',
 'b2015.446_H_CD3',
 'b2014.884_CD3',
 'b2013.10225_CD3',
 'B2008.365_CD3',
 'B2001.32133_CD3',
 'b2008.8850_CD3',
 'B2008.37930_CD3']

In [8]:
import os
import numpy as np
import shutil
import pandas as pd


In [9]:
import itertools
def train_test_split(output_dir):
    output_path = Path(output_dir)
    inputfile = output_path/'wsi.txt'
    path  = Path(inputfile)

    allWSI = (path).read_text().split('\n')
    train_ratio = 0.80

    np.random.seed(1923)
    np.random.shuffle(allWSI)
    train_WSI, val_WSI = np.split(np.array(allWSI),
                                             [int(len(allWSI) * train_ratio),
                                             ])
    valSlide = output_path/'valSlides.txt'
    trainSlide = output_path/'trainSlides.txt'
    Path(valSlide).touch()
    Path(trainSlide).touch()
    
    v = open(valSlide, 'w')
    t = open(trainSlide, 'w')
    
    for tra, val in itertools.zip_longest(train_WSI.tolist(), val_WSI.tolist()):
        print(val, file=v)
        print(tra, file=t)
    
    print("Done")

       
train_test_split(imageDir)

Done


In [10]:
output_path = Path(imageDir)
valSlide = output_path/'valSlides.txt'
trainSlide = output_path/'trainSlides.txt'
trainNames=[line.strip() for line in open(trainSlide)]
valNames=[line.strip() for line in open(valSlide)]

valTxt = os.path.join(imageDir,'valid.txt')
trainTxt = os.path.join(imageDir,'train.txt')
Path(valTxt).touch()
Path(trainTxt).touch()


In [12]:
vt = open(valTxt, 'w')
tt = open(trainTxt, 'w')
for imageFilename in allImages:
    #print(imageFilename)
    if imageFilename.startswith(tuple(trainNames)):
        print(imageFilename, file=tt)
        #print("Training: ",imageFilename)    
    elif imageFilename.startswith(tuple(valNames)):
        print(imageFilename, file=vt)
        #print("Validation: ",imageFilename)

At the end, check the files. Because of the lenght issue there might be some funny results.